# **Langchain Framework Hands-On #04**

In [1]:
import os
from langchain_openai import ChatOpenAI

my_key = os.environ['OPENAI_API_KEY']

### Simple LLM Chat Component

In [31]:
llm = ChatOpenAI(openai_api_key=my_key, model='gpt-3.5-turbo-1106', temperature=0.8)
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks to streamline the testing process. It can also assist in creating test cases, generating test data, and executing tests to identify and fix any issues or bugs in the software. Additionally, Langsmith can offer continuous integration and deployment solutions to ensure that testing is integrated into the development process and that any new changes are tested before being deployed to production. Overall, Langsmith can help improve the efficiency and effectiveness of testing efforts, ultimately leading to higher quality software.')

### LLM Chain

In [32]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [33]:
chain = prompt | llm 

In [34]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith can be a valuable tool for testing in several ways:\n\n1. Automated Testing: Langsmith can be used to develop automated tests for language processing components such as natural language understanding (NLU) models, sentiment analysis algorithms, or language generation systems. By creating test cases that simulate various inputs and expected outputs, developers can quickly validate the accuracy and performance of their language processing components.\n\n2. Regression Testing: When making changes to language processing components, it's essential to ensure that existing functionality remains intact. Langsmith can be utilized to run regression tests that verify that previous capabilities and behaviors are still functioning correctly after code modifications.\n\n3. Performance Testing: Langsmith can help in assessing the performance of language processing systems by generating a wide range of language inputs to measure response times, resource utilization, and sc

In [35]:
# Output of a ChatModel
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [36]:
chain = prompt | llm | output_parser

In [37]:
chain.invoke({"input": "how can langsmith help with testing?"})

"Langsmith can help with testing by providing a powerful and flexible testing framework for developers. It offers features such as unit testing, integration testing, and end-to-end testing capabilities, allowing developers to thoroughly test their code and ensure its quality and reliability.\n\nLangsmith's testing framework supports various testing methodologies and provides tools for creating and running test cases, managing test data, and reporting test results. It also integrates with popular testing libraries and tools, making it easy to incorporate Langsmith into existing testing workflows.\n\nAdditionally, Langsmith's built-in test automation features enable developers to automate repetitive testing tasks, saving time and effort in the testing process. This can help improve the efficiency of testing practices and increase the overall quality of the software being developed.\n\nOverall, Langsmith's testing capabilities can help developers streamline their testing processes, identi

### Retriever Chain : Similarity Search

In [42]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

embeddings = OpenAIEmbeddings()
raw_documents = TextLoader('./docs.txt').load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(raw_documents)
vector = FAISS.from_documents(documents, embeddings)

In [43]:
query = "What do you want to say?"
docs = vector.similarity_search(query)
print(docs[0].page_content)

Our challenges may be new. The instruments with which we meet them may be new. But those values upon which our success depends — honesty and hard work, courage and fair play, tolerance and curiosity, loyalty and patriotism — these things are old. These things are true. They have been the quiet force of progress throughout our history. What is demanded then is a return to these truths. What is required of us now is a new era of responsibility — a recognition, on the part of every American, that we have duties to ourselves, our nation, and the world — duties that we do not grudgingly accept but rather seize gladly, firm in the knowledge that there is nothing so satisfying to the spirit, so defining of our character, than giving our all to a difficult task.
This is the price and the promise of citizenship.
This is the source of our confidence — the knowledge that God calls on us to shape an uncertain destiny.
This is the meaning of our liberty and our creed — why men and women and childre

### Conversation Retriever Chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


# First we need a prompt that we can pass into an LLM to generate this search query
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)